In [ ]:
# supress any warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# configuring logging
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [ ]:
# python imports
import re
import numpy as np
import pandas as pd
from pprint import pprint

# third-party imports
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim

import matplotlib.pyplot as plt

In [ ]:
# loading english stopwords
stop_words = stopwords.words('english')

In [ ]:
# downloading spacy 'en_core_web_sm' model
# !python3 -m spacy download en_core_web_sm

# loading downloaded model en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

## Utils 

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
def preprocess_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

In [ ]:
def format_topics_sentences(ldamodel, corpus, texts):
    sent_topics_df = pd.DataFrame()

    # get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # get the sominant topic, perc contribution and keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = pd.concat([sent_topics_df, pd.DataFrame([[int(topic_num), round(prop_topic, 4), topic_keywords]])], ignore_index=True)
            else:
                break

    # add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df.columns = ['dominant_topic', 'perc_contribution', 'topic_keywords', 'prompt']
    
    return(sent_topics_df)

In [ ]:
def lda_grid_search(texts, id2word, corpus, num_topics, alphas, betas):

    best_lda_model = None
    best_coherence_lda = 0.0
    
    for n_topic in num_topics:
        for alpha in alphas:
            for beta in betas:
                lda_model = gensim.models.ldamodel.LdaModel(
                    corpus=corpus, id2word=id2word, num_topics=n_topic, random_state=100,
                    update_every=1, chunksize=100, passes=10, alpha=alpha, eta=beta, per_word_topics=True)
                
                coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
                coherence_lda = coherence_model_lda.get_coherence()

                if coherence_lda > best_coherence_lda:
                    best_lda_model = lda_model

    return best_lda_model

## Loading data

In [ ]:
data = pd.read_csv('data/prompt_injection_train.csv')

# filtering malicious prompts
data = data.loc[data['label'] == 1]

## Training LDA Model

In [ ]:
# convert a document into a list of lowercase tokens, removing 
# punctuations, and ignoring tokens that are too short or too long
data = data.prompt.values.tolist()
data_words = list(sent_to_words(data))

In [ ]:
# build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
trigram_mod = gensim.models.phrases.Phraser(trigram)

# remove Stopwords, form bigrams, trigrams and Lemmatization
texts = preprocess_words(data_words) 

In [ ]:
# create dictionary
id2word = corpora.Dictionary(texts)

# term document frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
# building LDA model
lda_model = lda_grid_search(texts=texts, 
                            id2word=id2word, 
                            corpus=corpus, 
                            num_topics=[2, 5, 7, 10, 12, 14], 
                            alphas=['symmetric', 0.3, 0.5, 0.7], 
                            betas=['auto', 0.3, 0.5, 0.7])

## Results

In [ ]:
perplexity = lda_model.log_perplexity(corpus)

coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence = coherence_model.get_coherence()

print(f'Perplexity: {perplexity}\nCoherence: {coherence}') 

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

In [ ]:
result = format_topics_sentences(lda_model, corpus, data)
result

In [ ]:
result['dominant_topic'].value_counts()